# Ayudantía 11: Capítulo 6 #
diego.herrerag00@uc.cl

## Importar librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from IPython.display import display

## Ejercicio Formativo 1

### Misión 1: lectura y exploración de datos

In [2]:
vehiculos = pd.read_csv('E1_datos.csv')
vehiculos.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2014,Kia,Forte,LX,Sedan,automatic,knafx4a64e5093418,nc,29.0,38339.0,red,black,avis corporation,11350.0,11500.0,Tue Feb 03 2015 01:15:00 GMT-0800 (PST)
1,2010,Audi,A4,2.0T Premium,Sedan,automatic,wauafafl4an066787,fl,46.0,46120.0,silver,black,lauderdale infiniti,14350.0,16600.0,Fri Dec 19 2014 09:35:00 GMT-0800 (PST)
2,2011,Toyota,Camry,LE,Sedan,automatic,4t1bf3ek3bu171593,ga,19.0,55958.0,gray,gray,capital one auto finance,10700.0,9000.0,Tue Jan 20 2015 01:30:00 GMT-0800 (PST)
3,2007,NaN,NaN,NaN,NaN,automatic,5tfrv54117x028139,ca,25.0,67044.0,gray,gray,e/e auto wholesale,14650.0,19500.0,Thu Jan 22 2015 04:00:00 GMT-0800 (PST)
4,2014,Kia,Forte,LX,Sedan,NaN,knafx4a66e5208858,ga,46.0,10386.0,silver,black,kia motors america inc,12850.0,12800.0,Thu Jan 08 2015 02:00:00 GMT-0800 (PST)


In [3]:
vehiculos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71779 entries, 0 to 71778
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          71779 non-null  int64  
 1   make          71178 non-null  object 
 2   model         71166 non-null  object 
 3   trim          71027 non-null  object 
 4   body          69869 non-null  object 
 5   transmission  63135 non-null  object 
 6   vin           71779 non-null  object 
 7   state         71779 non-null  object 
 8   condition     66451 non-null  float64
 9   odometer      71751 non-null  float64
 10  color         71662 non-null  object 
 11  interior      71662 non-null  object 
 12  seller        71779 non-null  object 
 13  mmr           71779 non-null  float64
 14  sellingprice  71779 non-null  float64
 15  saledate      71779 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 8.8+ MB


**Llave primaria (Primary Key):**

* vin: Identificador único de cada vehículo

**Llaves foráneas (Foreign Key):**

* make, de la relación Manufacturers

* model, de la relación Models

* state, de la relación States

* seller, de la relación Sellers

### Misión 2: modelación de entidades


. A continuación, se indican las tablas a modelar:

- **Vehicles**: Entidad principal de los vehículos, que incluye el año de fabricación, el VIN y detalles específicos como color, interior, condición, etc.
- **Manufacturers**: Almacena información de cada fabricante (make).
- **Models**: Incluye los modelos de vehículos, vinculados a cada fabricante.
- **States**: Contiene los estados de registro, permitiendo filtrar por ubicación.
- **Sellers**: Registra la información del vendedor de cada vehículo.
- **Sales**: Contiene la información de cada venta realizada, incluyendo el valor de mercado (mmr), precio de venta y fecha.

**Vehicles**

Atributos:

* vin (Primary Key)

* year, trim, body, transmission, condition, odometer, color, interior

* model_id (Foreign Key de Models)

* state_id (Foreign Key de States)

Relaciones:

* Pertenece a un modelo y un estado, por lo cual su relación es 1:1.

* Tiene una única venta asociada, por lo cual, su relación es 1:1 con Sales.

**Manufacturers**

Atributos:

* manufacturer_id (Primary Key)

* make (nombre del fabricante. Por ej: "Ford", "Toyota", etc.)

Relaciones:

* Un fabricante tiene muchos modelos, por lo tanto, su relación es 1:N.

**Models**

Atributos:

* model_id (Primary Key)

* model_name (ej. "Corolla")

* manufacturer_id (Foreign Key de Manufacturers)

* trim (designación adicional del modelo)

Relaciones:

* Un modelo pertenece a un fabricante, entonces, la relación es N:1.

* Un modelo puede tener muchos vehículos, entonces, la relación es 1:N.

**States**

Atributos:

* state_id (Primary Key)

* state (sigla del estado. ej: "ca", "ny", etc.)

Relaciones:

* Un estado puede tener muchos vehículos, entonces, la relación es 1:N.

**Sellers**

Atributos:

* seller_id (Primary Key)

* seller_name (nombre del vendedor)

Relaciones:

* Un vendedor puede estar asociado a muchas ventas, entonces, la relación es 1:N.

**Sales**

Atributos:

* sale_id (Primary Key)

* vin (Foreign Key de Vehicles)

* seller_id (Foreign Key de Sellers)

* mmr, sellingprice, saledate

Relaciones:

* Una venta está asociada a un vehículo y un vendedor, entonces la relación es 1:1.

### Misión 3: creación de tablas

In [4]:
connection = sqlite3.connect('vehicles.db')
cursor = connection.cursor()

Crear tabla Manufacturers.

In [5]:
cursor.execute("""
CREATE TABLE Manufacturers (
    manufacturer_id INTEGER PRIMARY KEY,
    make TEXT NOT NULL
);
""")

Crear tabla Models.

In [6]:
cursor.execute("""
CREATE TABLE Models (
    model_id INTEGER PRIMARY KEY,
    model_name TEXT NOT NULL,
    trim TEXT,
    manufacturer_id INTEGER,
    FOREIGN KEY (manufacturer_id) REFERENCES Manufacturers (manufacturer_id)
);
""")

Crear tabla States.

In [7]:
cursor.execute("""
CREATE TABLE States (
    state_id INTEGER PRIMARY KEY,
    state TEXT NOT NULL
);
""")

Crear tabla Sellers.

In [8]:
cursor.execute("""
CREATE TABLE Sellers (
    seller_id INTEGER PRIMARY KEY,
    seller_name TEXT NOT NULL
);
""")

Crear tabla Vehicles.

In [9]:
cursor.execute("""
CREATE TABLE Vehicles (
    vin TEXT PRIMARY KEY,
    year INTEGER,
    body TEXT,
    transmission TEXT,
    condition INTEGER,
    odometer INTEGER,
    color TEXT,
    interior TEXT,
    model_id INTEGER,
    state_id INTEGER,
    seller_id INTEGER,
    FOREIGN KEY (model_id) REFERENCES Models (model_id),
    FOREIGN KEY (state_id) REFERENCES States (state_id),
    FOREIGN KEY (seller_id) REFERENCES Sellers (seller_id)
);
""")

Crear tabla Sales.

In [10]:
cursor.execute("""
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY,
    vin TEXT,
    mmr REAL,
    sellingprice REAL,
    saledate DATE,
    FOREIGN KEY (vin) REFERENCES Vehicles (vin)
);
""")

Hacer commit de los cambios.

In [11]:
connection.commit()

### Misión 4: carga de datos en las tablas

In [12]:
vehiculos = vehiculos.dropna(subset=['make']) # Acá se eliminan los valores nulos de la columna make, antes de cargar los datos en la tabla Manufacturers.

Carga de datos en Manufacturers.

In [13]:
for make in vehiculos['make'].unique():
    cursor.execute("INSERT INTO Manufacturers (make) VALUES (?)", (make,))

Carga de datos en Models.

In [14]:
models = vehiculos.dropna(subset=['model', 'make'])

for _, fila in models.iterrows():
    cursor.execute("SELECT manufacturer_id FROM Manufacturers WHERE make = ?", (fila['make'],))
    manufacturer_id = cursor.fetchone()
    if manufacturer_id:
        manufacturer_id = manufacturer_id[0]
        cursor.execute("""
        INSERT INTO Models (model_name, trim, manufacturer_id)
        VALUES (?, ?, ?)
        """, (fila['model'], fila.get('trim', None), manufacturer_id))

Carga de datos en States.

In [15]:
vehiculos = vehiculos.dropna(subset=['state'])

for state in vehiculos['state'].unique():
    cursor.execute("INSERT OR IGNORE INTO States (state) VALUES (?)", (state,))

Carga de datos en Sellers.

In [16]:
vehiculos = vehiculos.dropna(subset=['seller'])

for seller_name in vehiculos['seller'].unique():
    cursor.execute("INSERT OR IGNORE INTO Sellers (seller_name) VALUES (?)", (seller_name,))

Cargar datos en Vehicles.

In [17]:
for _, fila in vehiculos.iterrows():
    cursor.execute("SELECT model_id FROM Models WHERE model_name = ?", (fila['model'],))
    model_id = cursor.fetchone()
    cursor.execute("SELECT state_id FROM States WHERE state = ?", (fila['state'],))
    state_id = cursor.fetchone()
    cursor.execute("SELECT seller_id FROM Sellers WHERE seller_name = ?", (fila['seller'],))
    seller_id = cursor.fetchone()

    if model_id and state_id and seller_id:
        model_id = model_id[0]
        state_id = state_id[0]
        seller_id = seller_id[0]
        cursor.execute("""
        INSERT OR IGNORE INTO Vehicles (vin, year, body, transmission, condition, odometer, color, interior, model_id, state_id, seller_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (fila['vin'], fila['year'], fila['body'], fila['transmission'], fila['condition'], fila['odometer'], fila['color'], fila['interior'], model_id, state_id, seller_id))

Cargar datos en Sales.

In [18]:
ventas = vehiculos.dropna(subset=['vin'])

for _, row in ventas.iterrows():
    cursor.execute("SELECT vin FROM Vehicles WHERE vin = ?", (row['vin'],))
    vin = cursor.fetchone()

    if vin:
        vin = vin[0]
        cursor.execute("""
        INSERT INTO Sales (vin, mmr, sellingprice, saledate)
        VALUES (?, ?, ?, ?)
        """, (vin, row['mmr'], row['sellingprice'], row['saledate']))

Hacer commit de los cambios.

In [19]:
connection.commit()

Mostrar número de registros cargados en cada tabla.

In [20]:
print("Número de registros en Manufacturers:", connection.execute("SELECT COUNT(*) FROM Manufacturers").fetchone()[0])
print("Número de registros en Models:", connection.execute("SELECT COUNT(*) FROM Models").fetchone()[0])
print("Número de registros en States:", connection.execute("SELECT COUNT(*) FROM States").fetchone()[0])
print("Número de registros en Sellers:", connection.execute("SELECT COUNT(*) FROM Sellers").fetchone()[0])
print("Número de registros en Vehicles:", connection.execute("SELECT COUNT(*) FROM Vehicles").fetchone()[0])
print("Número de registros en Sales:", connection.execute("SELECT COUNT(*) FROM Sales").fetchone()[0])

Número de registros en Manufacturers: 91
Número de registros en Models: 71166
Número de registros en States: 36
Número de registros en Sellers: 6025
Número de registros en Vehicles: 70967
Número de registros en Sales: 71166


Cerrar la conexión.

In [21]:
connection.close()

## Ejercicio Formativo 2

In [22]:
connection = sqlite3.connect('vehicles.db')
cursor = connection.cursor()

Crear una función para obtener resultados de consultas.

In [23]:
def ejecutar(consulta):
    cursor.execute(consulta)
    resultado = cursor.fetchall()
    return resultado

### 1. Liste todos los fabricantes junto con la cantidad de vehículos asociados.

In [24]:
consulta = """
SELECT Mf.make as fabricante, COUNT(V.vin) as cantidad
FROM Manufacturers Mf
JOIN Models Md ON Mf.manufacturer_id = Md.manufacturer_id
JOIN Vehicles V ON Md.model_id = V.model_id
GROUP BY Mf.manufacturer_id
ORDER BY COUNT(V.vin) DESC
"""

resultado = ejecutar(consulta)

print("Fabricante | Cantidad de vehículos \n")
for fabricante, cantidad in resultado:
    print(f'{fabricante}: {cantidad:}')

Fabricante | Cantidad de vehículos 

Ford: 11634
Chevrolet: 7559
Nissan: 6445
Toyota: 4932
Honda: 4013
Dodge: 3814
Hyundai: 2798
BMW: 2742
Chrysler: 2286
Kia: 2245
Mercedes-Benz: 2228
Jeep: 1953
Infiniti: 1860
Volkswagen: 1624
Lexus: 1519
GMC: 1259
Mazda: 1141
Cadillac: 1053
Acura: 884
Subaru: 834
Audi: 741
Lincoln: 730
Pontiac: 663
Buick: 653
Mitsubishi: 545
chevrolet: 531
Volvo: 496
Saturn: 431
MINI: 386
Mercury: 284
Scion: 223
Land Rover: 222
Jaguar: 217
ford: 197
Porsche: 164
Suzuki: 161
Ram: 142
dodge: 139
HUMMER: 106
chrysler: 102
Saab: 93
FIAT: 91
land rover: 74
mazda: 63
honda: 57
Oldsmobile: 57
lexus: 56
mitsubishi: 54
cadillac: 53
toyota: 48
jeep: 41
smart: 35
bmw: 31
Isuzu: 26
mercedes: 26
nissan: 26
subaru: 24
vw: 14
acura: 12
Maserati: 12
volkswagen: 12
chev truck: 11
Bentley: 11
pontiac: 10
buick: 9
oldsmobile: 9
hyundai: 8
mercury: 8
lincoln: 7
gmc: 6
kia: 3
gmc truck: 3
plymouth: 2
Rolls-Royce: 2
maserati: 2
Ferrari: 2
Tesla: 2
landrover: 2
Plymouth: 2
Daewoo: 1
Fisker:

### 2. Encuentre todos los vehículos registrados en un estado específico.

In [25]:
estado = 'ca'

consulta = f"""
SELECT V.vin as vin
FROM Vehicles V
JOIN Models Md ON V.model_id = Md.model_id
JOIN States S ON V.state_id = S.state_id
WHERE S.state = '{estado}'
"""

resultado = ejecutar(consulta)

print(f"Vehículos registrados en {estado}: \n")
for vin in resultado:
    print(vin[0])

Vehículos registrados en ca: 

2hgfb2f53ch334060
wbaaz33424kp86563
sajwa2gb7blv17938
jtegp21a350058734
jn1cv6ap4dm714714
wddng7bb8aa331855
wbaeh73434b190048
sajwa1cb1blv16851
1nxbu4ee0az336682
3gyek63nx3g231752
1fadp3e2xdl131942
jn8as5mv9bw690787
kndmb233366037319
5nmsh13e07h027160
1c6rr6gt3es236838
nm0ls7dn3ct084479
jt2bg12k3t0390856
wbanb53527cp04033
waujc68e75a091829
5tdza23c25s273936
3fa6p0h71dr291594
1gcgg25v561109079
3fadp4cj6cm126590
scbcr63w45c028141
3a8fy48bx8t163116
wdbkk47f9yf163449
3vw4a7aj5dm411439
1gker23788j245056
1ftpw14526fa13203
wbanw135x8cz81382
2c4rc1cg8dr805432
1fahp2hw5cg109638
kndmg4c71c6479754
1n4al2ap9cc119516
3lnhl2gc1cr814700
4t4bf3ek3br177416
jn1cv6ek5bm212192
1fahp2f84dg193120
1n4aa5apxdc839917
wmwmf73558tv38670
2hnyd2h33ch514517
1fbss31l69da25783
wvwrh63b82p365195
1gchk29u81e187989
1d8gt28k48w233834
3vwdp7aj2cm077383
jthbd192140092750
3fahp0ha7cr259543
jn8az08t26w420555
1fm5k8f88fga20956
1g1za5e19bf133917
3fa6p0h76dr291638
1n4al2ap0cc122059
wddng7gb8aa3572

### 3. Calcule el precio promedio de venta por tipo de carrocería.

In [26]:
consulta = """
SELECT Mf.make as fabricante, COUNT(V.vin) as cantidad
FROM Manufacturers Mf
JOIN Models Md ON Mf.manufacturer_id = Md.manufacturer_id
JOIN Vehicles V ON Md.model_id = V.model_id
GROUP BY Mf.manufacturer_id
ORDER BY COUNT(V.vin) DESC
"""

resultado = ejecutar(consulta)

print("Fabricante | Cantidad de vehículos \n")
for fabricante, cantidad in resultado:
    print(f'{fabricante}: {cantidad}')

Fabricante | Cantidad de vehículos 

Ford: 11634
Chevrolet: 7559
Nissan: 6445
Toyota: 4932
Honda: 4013
Dodge: 3814
Hyundai: 2798
BMW: 2742
Chrysler: 2286
Kia: 2245
Mercedes-Benz: 2228
Jeep: 1953
Infiniti: 1860
Volkswagen: 1624
Lexus: 1519
GMC: 1259
Mazda: 1141
Cadillac: 1053
Acura: 884
Subaru: 834
Audi: 741
Lincoln: 730
Pontiac: 663
Buick: 653
Mitsubishi: 545
chevrolet: 531
Volvo: 496
Saturn: 431
MINI: 386
Mercury: 284
Scion: 223
Land Rover: 222
Jaguar: 217
ford: 197
Porsche: 164
Suzuki: 161
Ram: 142
dodge: 139
HUMMER: 106
chrysler: 102
Saab: 93
FIAT: 91
land rover: 74
mazda: 63
honda: 57
Oldsmobile: 57
lexus: 56
mitsubishi: 54
cadillac: 53
toyota: 48
jeep: 41
smart: 35
bmw: 31
Isuzu: 26
mercedes: 26
nissan: 26
subaru: 24
vw: 14
acura: 12
Maserati: 12
volkswagen: 12
chev truck: 11
Bentley: 11
pontiac: 10
buick: 9
oldsmobile: 9
hyundai: 8
mercury: 8
lincoln: 7
gmc: 6
kia: 3
gmc truck: 3
plymouth: 2
Rolls-Royce: 2
maserati: 2
Ferrari: 2
Tesla: 2
landrover: 2
Plymouth: 2
Daewoo: 1
Fisker:

### 4. Liste los vehículos con la mayor diferencia entre el precio de venta y el valor de mercado (MMR) ordenados de mayor a menor.

In [27]:
consulta = """
SELECT V.vin as vin, S.sellingprice as precio, S.mmr as valor, S.sellingprice - S.mmr as diferencia
FROM Sales S
JOIN Vehicles V ON S.vin = V.vin
ORDER BY diferencia DESC
LIMIT 10
"""

resultado = ejecutar(consulta)

print("       VIN        |  Precio  |   Valor  | Diferencia \n")
for vin, precio, valor, diferencia in resultado:
    print(f'{vin}: {precio:8} | {valor:8} | {diferencia:9}')

       VIN        |  Precio  |   Valor  | Diferencia 

wd3pf4cd5c5624350:  71000.0 |  29500.0 |   41500.0
1gnskjkc4fr158207:  46400.0 |  11100.0 |   35300.0
4f2yz04136km20229:  35000.0 |   8500.0 |   26500.0
wddug8cb8ea054954: 106400.0 |  83000.0 |   23400.0
3gcukrec8eg233825:  36250.0 |  14200.0 |   22050.0
3fadp4bj5em142820:  30900.0 |   9850.0 |   21050.0
2gnalcek7e6105115:  38000.0 |  17650.0 |   20350.0
1n4al3ap9ec119301:  37200.0 |  16900.0 |   20300.0
3fa6p0h73dr107451:  34500.0 |  14700.0 |   19800.0
1gnskje71er203407:  37200.0 |  17500.0 |   19700.0


### 5. Identifique los estados con el mayor número de ventas y el precio promedio más alto.

In [28]:
consulta = """
SELECT St.state as estado, COUNT(S.sale_id) as cantidad
FROM Sales S
JOIN Vehicles V ON S.vin = V.vin
JOIN States St ON V.state_id = St.state_id
GROUP BY St.state_id
ORDER BY cantidad DESC
LIMIT 5
"""

resultado = ejecutar(consulta)

print("Estado | Cantidad de ventas \n")
for estado, cantidad in resultado:
    print(f'{estado}: {cantidad}')

Estado | Cantidad de ventas 

ca: 11038
fl: 9115
pa: 6730
tx: 5635
ga: 4290


In [29]:
consulta_2 = """
SELECT St.state as estado, ROUND(AVG(S.sellingprice), 2) as precio_promedio
FROM Sales S
JOIN Vehicles V ON S.vin = V.vin
JOIN States St ON V.state_id = St.state_id
GROUP BY St.state_id
ORDER BY precio_promedio DESC
LIMIT 5
"""

resultado_2 = ejecutar(consulta_2)

print("Estado | Precio promedio \n")
for estado, precio_promedio in resultado_2:
    print(f'{estado}: {precio_promedio}')

Estado | Precio promedio 

tn: 16889.25
ms: 16166.67
on: 15506.22
pa: 15497.08
co: 14977.21


### 6. Encuentre el fabricante con el mayor ingreso total por ventas.

In [30]:
consulta = """
SELECT Mf.make as fabricante, SUM(S.sellingprice) as ingreso_total
FROM Manufacturers Mf
JOIN Models Md ON Mf.manufacturer_id = Md.manufacturer_id
JOIN Vehicles V ON Md.model_id = V.model_id
JOIN Sales S ON V.vin = S.vin
GROUP BY Mf.manufacturer_id
ORDER BY ingreso_total DESC
LIMIT 1
"""

resultado = ejecutar(consulta)

print("Fabricante con mayor ingreso total por ventas \n")
for fabricante, ingreso_total in resultado:
    print(f'{fabricante}: {ingreso_total}')

Fabricante con mayor ingreso total por ventas 

Ford: 161356093.0


Cerrar la conexión.

In [31]:
connection.close()

## Ejercicio Formativo 3


### Misión 1: Exploración inicial de los datos


Exploramos el archivo brands.csv


In [32]:
brands = pd.read_csv('E3_datos/brands.csv')
print(f"Registros: {len(brands)}, Columnas: {len(brands.columns)}")
brands.head()
print("Columnas:", brands.columns.tolist())


Registros: 9, Columnas: 2
Columnas: ['brand_id', 'brand_name']


In [33]:
brands = pd.read_csv('E3_datos/brands.csv')
print(f"Registros: {len(brands)}, Columnas: {len(brands.columns)}")
display(brands.head()) # Clave el display si quieren que también se vea las primeras filas del data set dentro del mismo output de la celda de codigo
print("Columnas:", brands.columns.tolist())


Registros: 9, Columnas: 2


,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey


Columnas: ['brand_id', 'brand_name']


Analizemos el data set del archivo brands.csv:
- **Llave primaria (Primary Key):** brand_id, que corresponde al identificador único de cada marca. 
- **Atributos:** brand_name, que corresponde al nombre de la marca
- **Relaciones:** Una marca puede tener muchos productos, lo cual sería una relación del tipo **1:N** con Products. 


Exploramos el archivo categories.csv


In [34]:
categories = pd.read_csv('E3_datos/categories.csv')
print(f"Registros: {len(categories)}, Columnas: {len(categories.columns)}")
display(categories.head())
print("Columnas:", categories.columns.tolist())


Registros: 7, Columnas: 2


,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,5,Electric Bikes


Columnas: ['category_id', 'category_name']


Analicemos el data set del archivo categories.csv:
- **Llave primaria (Primary Key):** category_id, identificador único de cada categoría
- **Atributos:** category_name, nombre de la categoría. 
- **Relaciones:** Una categoría puede tener muchos productos, entonces la relación que tiene con Products es **1:N**.


Exploramos el archivo products.csv


In [35]:
products = pd.read_csv('E3_datos/products.csv')
print(f"Registros: {len(products)}, Columnas: {len(products.columns)}")
display(products.head())
print("Columnas:", products.columns.tolist())

Registros: 321, Columnas: 6


,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99


Columnas: ['product_id', 'product_name', 'brand_id', 'category_id', 'model_year', 'list_price']


Analicemos el data set del archivo products.csv:
- **Llave primaria (Primary Key):** product_id (identificador único de cada producto)
- **Atributos:** product_name, model_year, list_price
- **Llaves foráneas (Foreign Key):** brand_id es el id de la relación Brands, category_id es el id de la relación Categories
- **Relaciones:** 
  - Un producto pertenece a una marca y una categoría (N:1 con Brands y Categories)
  - Un producto puede aparecer en muchos items de orden (1:N con Order_Items)
  - Un producto puede tener stock en múltiples tiendas (1:N con Stocks)


Exploramos el archivo customers.csv


In [36]:
customers = pd.read_csv('E3_datos/customers.csv')
print(f"Registros: {len(customers)}, Columnas: {len(customers.columns)}")
display(customers.head())
print("Columnas:", customers.columns.tolist())


Registros: 1445, Columnas: 9


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


Columnas: ['customer_id', 'first_name', 'last_name', 'phone', 'email', 'street', 'city', 'state', 'zip_code']


Analicemos el data set del archivo customers.csv:
- **Llave primaria (Primary Key):** customer_id (identificador único de cada cliente)
- **Atributos:** first_name, last_name, phone, email, street, city, state, zip_code
- **Relaciones:** Un cliente puede realizar muchas órdenes (1:N con Orders)


Exploramos el archivo stores.csv


In [37]:
stores = pd.read_csv('E3_datos/stores.csv')
print(f"Registros: {len(stores)}, Columnas: {len(stores.columns)}")
print("Primeras filas:")
display(stores.head())
print("Columnas:", stores.columns.tolist())


Registros: 3, Columnas: 8
Primeras filas:


,store_id,store_name,phone,email,street,city,state,zip_code
0,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
1,2,Baldwin Bikes,(516) 379-8888,baldwin@bikes.shop,4200 Chestnut Lane,Baldwin,NY,11432
2,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088


Columnas: ['store_id', 'store_name', 'phone', 'email', 'street', 'city', 'state', 'zip_code']


Analicemos el data set del archivo stores.csv:
- **Llave primaria (Primary Key):** store_id (identificador único de cada tienda)
- **Atributos:** store_name, phone, email, street, city, state, zip_code
- **Relaciones:** 
  - Una tienda puede tener muchos empleados (1:N con Staffs)
  - Una tienda puede tener muchas órdenes (1:N con Orders)
  - Una tienda puede tener stock de muchos productos (1:N con Stocks)


Exploramos el archivo staffs.csv


In [38]:
staffs = pd.read_csv('E3_datos/staffs.csv')
print(f"Registros: {len(staffs)}, Columnas: {len(staffs.columns)}")
display(staffs.head())
print("Columnas:", staffs.columns.tolist())


Registros: 10, Columnas: 8


,staff_id,first_name,last_name,email,phone,active,store_id,manager_id
0,1,Fabiola,Jackson,fabiola.jackson@bikes.shop,(831) 555-5554,1,1,NaN
1,2,Mireya,Copeland,mireya.copeland@bikes.shop,(831) 555-5555,1,1,1.0
2,3,Genna,Serrano,genna.serrano@bikes.shop,(831) 555-5556,1,1,2.0
3,4,Virgie,Wiggins,virgie.wiggins@bikes.shop,(831) 555-5557,1,1,2.0
4,5,Jannette,David,jannette.david@bikes.shop,(516) 379-4444,1,2,1.0


Columnas: ['staff_id', 'first_name', 'last_name', 'email', 'phone', 'active', 'store_id', 'manager_id']


Analicemos el data set del archivo staffs.csv:
- **Llave primaria (Primary Key):** staff_id (identificador único de cada empleado)
- **Atributos:** first_name, last_name, email, phone, active
- **Llaves foráneas (Foreign Key):** store_id de Stores, manager_id de Staffs
- **Relaciones:** 
  - Un empleado pertenece a una tienda (N:1 con Stores)
  - Un empleado puede tener un manager (N:1 con Staffs, auto-referencia)
  - Un empleado puede procesar muchas órdenes (1:N con Orders)


Exploramos el archivo orders.csv


In [39]:
orders = pd.read_csv('E3_datos/orders.csv')
print(f"Registros: {len(orders)}, Columnas: {len(orders.columns)}")
print("\nPrimeras filas:")
display(orders.head())
print("\nColumnas:", orders.columns.tolist())
print("\nValores únicos de order_status:", orders['order_status'].unique())


Registros: 1615, Columnas: 8

Primeras filas:


,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6



Columnas: ['order_id', 'customer_id', 'order_status', 'order_date', 'required_date', 'shipped_date', 'store_id', 'staff_id']

Valores únicos de order_status: [4 3 2 1]


Analicemos el data set del archivo orders.csv:
- **Llave primaria (Primary Key):** order_id (identificador único de cada orden)
- **Atributos:** order_status, order_date, required_date, shipped_date
- **Llaves foráneas (Foreign Key):** customer_id de Customers, store_id de Stores, staff_id de Staffs
- **Relaciones:** 
  - Una orden pertenece a un cliente, una tienda y un empleado, entonces, la relación va a ser N:1 con Customers, Stores y Staffs
  - Una orden puede tener muchos items, entonces la relación va a ser 1:N con Order_Items.


Explorar archivo order_items.csv


In [40]:
order_items = pd.read_csv('E3_datos/order_items.csv')
print(f"Registros: {len(order_items)}, Columnas: {len(order_items.columns)}")
print("\nPrimeras filas:")
display(order_items.head())
print("\nColumnas:", order_items.columns.tolist())


Registros: 4722, Columnas: 6

Primeras filas:


,order_id,item_id,product_id,quantity,list_price,discount
0,1,1,20,1,599.99,0.20
1,1,2,8,2,1799.99,0.07
2,1,3,10,2,1549.00,0.05
3,1,4,16,2,599.99,0.05
4,1,5,4,1,2899.99,0.20



Columnas: ['order_id', 'item_id', 'product_id', 'quantity', 'list_price', 'discount']


Analicemos el data set del archivo order_items.csv:
- **Llave primaria compuesta (Primary Key):** (order_id, item_id), ambos identifican de forma única cada item en una orden
- **Atributos:** quantity, list_price, discount
- **Llaves foráneas (Foreign Key):** order_id de la relación  Orders, product_id de la relación Products
- **Relaciones:** 
  - Un item pertenece a una orden y un producto, entonces, la relación es de N:1 con Orders y Products. 


Exploramos el archivo stocks.csv


In [41]:
stocks = pd.read_csv('E3_datos/stocks.csv')
print(f"Registros: {len(stocks)}, Columnas: {len(stocks.columns)}")
print("\nPrimeras filas:")
display(stocks.head())
print("\nColumnas:", stocks.columns.tolist())


Registros: 939, Columnas: 3

Primeras filas:


,store_id,product_id,quantity
0,1,1,27
1,1,2,5
2,1,3,6
3,1,4,23
4,1,5,22



Columnas: ['store_id', 'product_id', 'quantity']


Analicemos el data set del archivo stocks.csv:
- **Llave primaria compuesta (Primary Key):** (store_id, product_i), ambos identifican de forma única el stock de un producto en una tienda
- **Atributos:** quantity (cantidad en stock)
- **Llaves foráneas (Foreign Key):** store_id de la relación Stores, product_id de la relación Products
- **Relaciones:** 
  - Un registro de stock pertenece a una tienda y un producto, entonces la relación es N:1 con Stores y Products.


### Misión 2: Propuesta de estructura relacional


**Brands**
- **Propósito:** Almacenar información de las marcas de productos
- **Atributos:**
  - brand_id (INTEGER, Primary Key, NOT NULL)
  - brand_name (TEXT, NOT NULL, UNIQUE)
- **Restricciones:** brand_id es llave primaria, brand_name debe ser único y no nulo

**Categories**
- **Propósito:** Almacenar información de las categorías de productos
- **Atributos:**
  - category_id (INTEGER, Primary Key, NOT NULL)
  - category_name (TEXT, NOT NULL, UNIQUE)
- **Restricciones:** category_id es llave primaria, category_name debe ser único y no nulo

**Products**
- **Propósito:** Almacenar información de los productos
- **Atributos:**
  - product_id (INTEGER, Primary Key, NOT NULL)
  - product_name (TEXT, NOT NULL)
  - brand_id (INTEGER, Foreign Key(Brands), NOT NULL)
  - category_id (INTEGER, Foreign Key (Categories), NOT NULL)
  - model_year (INTEGER)
  - list_price (REAL, NOT NULL, CHECK(list_price >= 0))
- **Restricciones:** product_id es llave primaria, brand_id y category_id son llaves foráneas, list_price debe ser no negativo

**Customers**
- **Propósito:** Almacenar información de los clientes
- **Atributos:**
  - customer_id (INTEGER, Primary Key, NOT NULL)
  - first_name (TEXT, NOT NULL)
  - last_name (TEXT, NOT NULL)
  - phone (TEXT)
  - email (TEXT)
  - street (TEXT)
  - city (TEXT)
  - state (TEXT)
  - zip_code (TEXT)
- **Restricciones:** customer_id es llave primaria, first_name y last_name no pueden ser nulos

**Stores**
- **Propósito:** Almacenar información de las tiendas
- **Atributos:**
  - store_id (INTEGER, Primary Key, NOT NULL)
  - store_name (TEXT, NOT NULL)
  - phone (TEXT)
  - email (TEXT)
  - street (TEXT)
  - city (TEXT)
  - state (TEXT)
  - zip_code (TEXT)
- **Restricciones:** store_id es llave primaria, store_name no puede ser nulo

**Staffs**
- **Propósito:** Almacenar información de los empleados
- **Atributos:**
  - staff_id (INTEGER, Primary Key, NOT NULL)
  - first_name (TEXT, NOT NULL)
  - last_name (TEXT, NOT NULL)
  - email (TEXT)
  - phone (TEXT)
  - active (INTEGER, NOT NULL, CHECK(active IN (0, 1)))
  - store_id (INTEGER, Foreign Key (Stores), NOT NULL)
  - manager_id (INTEGER, Foreign Key (Staffs))
- **Restricciones:** staff_id es llave primaria, store_id es llave foránea, manager_id es llave foránea auto-referencial, active debe ser 0 o 1

**Orders**
- **Propósito:** Almacenar información de las órdenes
- **Atributos:**
  - order_id (INTEGER, PK, NOT NULL)
  - customer_id (INTEGER, Foreign Key (Customers), NOT NULL)
  - order_status (INTEGER, NOT NULL, CHECK(order_status BETWEEN 1 AND 4))
  - order_date (DATE, NOT NULL)
  - required_date (DATE)
  - shipped_date (DATE)
  - store_id (INTEGER, Foreign Key (Store), NOT NULL)
  - staff_id (INTEGER, Foreign Key (Staffs), NOT NULL)
- **Restricciones:** order_id es llave primaria, customer_id, store_id y staff_id son llaves foráneas, order_status debe estar entre 1 y 4, order_date no puede ser nulo

**Order_Items**
- **Propósito:** Almacenar información de los items de cada orden
- **Atributos:**
  - order_id (INTEGER, Foreign Key (Orders), NOT NULL)
  - item_id (INTEGER, NOT NULL)
  - product_id (INTEGER, Foreign Key (Products), NOT NULL)
  - quantity (INTEGER, NOT NULL, CHECK(quantity > 0))
  - list_price (REAL, NOT NULL, CHECK(list_price >= 0))
  - discount (REAL, NOT NULL, CHECK(discount >= 0 AND discount <= 1))
- **Restricciones:** (order_id, item_id) es llave primaria compuesta, order_id y product_id son llaves foráneas, quantity debe ser positiva, list_price y discount deben ser no negativos, discount debe estar entre 0 y 1

**Stocks**
- **Propósito:** Almacenar información del inventario de productos por tienda
- **Atributos:**
  - store_id (INTEGER, Foreign Key (Stores), NOT NULL)
  - product_id (INTEGER, Foreign Key (Products), NOT NULL)
  - quantity (INTEGER, NOT NULL, CHECK(quantity >= 0))
- **Restricciones:** (store_id, product_id) es llave primaria compuesta, store_id y product_id son llaves foráneas, quantity debe ser no negativa


### Misión 3: Implementación del esquema en SQLite


In [42]:
# Primero que todo, se crea la conexión con la base de datos.
connection = sqlite3.connect('store.db') # Acá se crea la conexión con la base de datos. En caso de que no exista el archivo, se creará, y en caso de que si exista, se conectará a él.
cursor = connection.cursor() # Acá se crea el cursor, que nos permitirá ejecutar las sentencias SQL.


Crear tabla Brands.


In [43]:
cursor.execute("""
CREATE TABLE Brands (
    brand_id INTEGER PRIMARY KEY,
    brand_name TEXT NOT NULL UNIQUE
);
""")


### Creación de Relaciones

Creamos la tabla Categories.


In [44]:
cursor.execute("""
CREATE TABLE Categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT NOT NULL UNIQUE
);
""")


Creamos la tabla Products.


In [45]:
cursor.execute("""
CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    brand_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    model_year INTEGER,
    list_price REAL NOT NULL CHECK(list_price >= 0),
    FOREIGN KEY (brand_id) REFERENCES Brands (brand_id),
    FOREIGN KEY (category_id) REFERENCES Categories (category_id)
);
""")


Creamos la tabla Customers.

In [46]:
cursor.execute("""
CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    phone TEXT,
    email TEXT,
    street TEXT,
    city TEXT,
    state TEXT,
    zip_code TEXT
);
""")


Creamos la tabla Stores.


In [47]:
cursor.execute("""
CREATE TABLE Stores (
    store_id INTEGER PRIMARY KEY,
    store_name TEXT NOT NULL,
    phone TEXT,
    email TEXT,
    street TEXT,
    city TEXT,
    state TEXT,
    zip_code TEXT
);
""")


Creamos la tabla Staffs.


In [48]:
cursor.execute("""
CREATE TABLE Staffs (
    staff_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT,
    phone TEXT,
    active INTEGER NOT NULL CHECK(active IN (0, 1)),
    store_id INTEGER NOT NULL,
    manager_id INTEGER,
    FOREIGN KEY (store_id) REFERENCES Stores (store_id),
    FOREIGN KEY (manager_id) REFERENCES Staffs (staff_id)
);
""")


Creamos la tabla Orders.


In [49]:
cursor.execute("""
CREATE TABLE Orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_status INTEGER NOT NULL CHECK(order_status BETWEEN 1 AND 4),
    order_date DATE NOT NULL,
    required_date DATE,
    shipped_date DATE,
    store_id INTEGER NOT NULL,
    staff_id INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES Customers (customer_id),
    FOREIGN KEY (store_id) REFERENCES Stores (store_id),
    FOREIGN KEY (staff_id) REFERENCES Staffs (staff_id)
);
""")


Creamos la tabla Order_Items.


In [50]:
cursor.execute("""
CREATE TABLE Order_Items (
    order_id INTEGER NOT NULL,
    item_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL CHECK(quantity > 0),
    list_price REAL NOT NULL CHECK(list_price >= 0),
    discount REAL NOT NULL CHECK(discount >= 0 AND discount <= 1),
    PRIMARY KEY (order_id, item_id),
    FOREIGN KEY (order_id) REFERENCES Orders (order_id),
    FOREIGN KEY (product_id) REFERENCES Products (product_id)
);
""")


Creamos la tabla Stocks.


In [51]:
cursor.execute("""
CREATE TABLE Stocks (
    store_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL CHECK(quantity >= 0),
    PRIMARY KEY (store_id, product_id),
    FOREIGN KEY (store_id) REFERENCES Stores (store_id),
    FOREIGN KEY (product_id) REFERENCES Products (product_id)
);
""")


Hacemos commit de los cambios para que se guarden los cambios en la base de datos.

In [52]:
connection.commit()


### Misión 4: Carga de datos y verificación


Tal como se nos solicita, cargamos los datos almacenaos en los distintos archivos csv para completar la información en las relaciones que acabamos de crear. 

Carga de datos en Brands.


In [53]:
brands_df = pd.read_csv('E3_datos/brands.csv')
for _, fila in brands_df.iterrows(): # Iteramos sobre cada fila del dataframe.
    cursor.execute("INSERT INTO Brands (brand_id, brand_name) VALUES (?, ?)", 
                   (int(fila['brand_id']), fila['brand_name'])) # Ejecutamos la sentencia SQL para insertar los datos en la tabla Brands.
connection.commit()

Carga de datos en Categories.


In [54]:
categories_df = pd.read_csv('E3_datos/categories.csv')
for _, fila in categories_df.iterrows():
    cursor.execute("INSERT INTO Categories (category_id, category_name) VALUES (?, ?)", 
                   (int(fila['category_id']), fila['category_name']))
connection.commit()

Carga de datos en Products.


In [55]:
products_df = pd.read_csv('E3_datos/products.csv')
for _, fila in products_df.iterrows():
    cursor.execute("""
        INSERT INTO Products (product_id, product_name, brand_id, category_id, model_year, list_price)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (int(fila['product_id']), fila['product_name'], 
          int(fila['brand_id']), int(fila['category_id']), 
          int(fila['model_year']),
          float(fila['list_price'])))
connection.commit()

Carga de datos en Customers. Para este caso, tomaremos en cuenta que los datos de las columnas pueden ser nulos, por lo que se debe verificar si el dato es nulo para no insertarlo.


In [56]:
customers_df = pd.read_csv('E3_datos/customers.csv')
for _, fila in customers_df.iterrows():
    cursor.execute("""
        INSERT INTO Customers (customer_id, first_name, last_name, phone, email, street, city, state, zip_code)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (int(fila['customer_id']), fila['first_name'], fila['last_name'],
          fila['phone'] if pd.notna(fila['phone']) else None,
          fila['email'] if pd.notna(fila['email']) else None,
          fila['street'] if pd.notna(fila['street']) else None,
          fila['city'] if pd.notna(fila['city']) else None,
          fila['state'] if pd.notna(fila['state']) else None,
          fila['zip_code'] if pd.notna(fila['zip_code']) else None))
connection.commit()

Carga de datos en Stores. Hacemos lo mismo que antes.


In [57]:
stores_df = pd.read_csv('E3_datos/stores.csv')
for _, fila in stores_df.iterrows():
    cursor.execute("""
        INSERT INTO Stores (store_id, store_name, phone, email, street, city, state, zip_code)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (int(fila['store_id']), fila['store_name'],
          fila['phone'] if pd.notna(fila['phone']) else None,
          fila['email'] if pd.notna(fila['email']) else None,
          fila['street'] if pd.notna(fila['street']) else None,
          fila['city'] if pd.notna(fila['city']) else None,
          fila['state'] if pd.notna(fila['state']) else None,
          fila['zip_code'] if pd.notna(fila['zip_code']) else None))
connection.commit()

Carga de datos en Staff.

In [58]:
staffs_df = pd.read_csv('E3_datos/staffs.csv')
for _, fila in staffs_df.iterrows():
    cursor.execute("""
        INSERT INTO Staffs (staff_id, first_name, last_name, email, phone, active, store_id, manager_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (int(fila['staff_id']), fila['first_name'], fila['last_name'],
          fila['email'] if pd.notna(fila['email']) else None,
          fila['phone'] if pd.notna(fila['phone']) else None,
          int(fila['active']),
          int(fila['store_id']),
          int(fila['manager_id']) if pd.notna(fila['manager_id']) else None))
connection.commit()

Carga de datos en Orders.


In [59]:
orders_df = pd.read_csv('E3_datos/orders.csv')
for _, fila in orders_df.iterrows():
    cursor.execute("""
        INSERT INTO Orders (order_id, customer_id, order_status, order_date, required_date, shipped_date, store_id, staff_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (int(fila['order_id']), int(fila['customer_id']), int(fila['order_status']),
          fila['order_date'], 
          fila['required_date'] if pd.notna(fila['required_date']) else None,
          fila['shipped_date'] if pd.notna(fila['shipped_date']) else None,
          int(fila['store_id']), int(fila['staff_id'])))
connection.commit()

Carga de datos en Order_Items.


In [60]:
order_items_df = pd.read_csv('E3_datos/order_items.csv')
for _, fila in order_items_df.iterrows():
    cursor.execute("""
        INSERT INTO Order_Items (order_id, item_id, product_id, quantity, list_price, discount)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (int(fila['order_id']), int(fila['item_id']), int(fila['product_id']),
          int(fila['quantity']), float(fila['list_price']), float(fila['discount'])))
connection.commit()

Carga de datos en Stocks.


In [61]:
stocks_df = pd.read_csv('E3_datos/stocks.csv')
for _, fila in stocks_df.iterrows():
    cursor.execute("""
        INSERT INTO Stocks (store_id, product_id, quantity)
        VALUES (?, ?, ?)
    """, (int(fila['store_id']), int(fila['product_id']), int(fila['quantity'])))
connection.commit()

Finalmente, debemos mostrar el número de registros cargados en cada una de las tablas creadas.


In [62]:
print("Número de registros en Brands:", connection.execute("SELECT COUNT(*) FROM Brands").fetchone()[0])
print("Número de registros en Categories:", connection.execute("SELECT COUNT(*) FROM Categories").fetchone()[0])
print("Número de registros en Products:", connection.execute("SELECT COUNT(*) FROM Products").fetchone()[0])
print("Número de registros en Customers:", connection.execute("SELECT COUNT(*) FROM Customers").fetchone()[0])
print("Número de registros en Stores:", connection.execute("SELECT COUNT(*) FROM Stores").fetchone()[0])
print("Número de registros en Staffs:", connection.execute("SELECT COUNT(*) FROM Staffs").fetchone()[0])
print("Número de registros en Orders:", connection.execute("SELECT COUNT(*) FROM Orders").fetchone()[0])
print("Número de registros en Order_Items:", connection.execute("SELECT COUNT(*) FROM Order_Items").fetchone()[0])
print("Número de registros en Stocks:", connection.execute("SELECT COUNT(*) FROM Stocks").fetchone()[0])


Número de registros en Brands: 9
Número de registros en Categories: 7
Número de registros en Products: 321
Número de registros en Customers: 1445
Número de registros en Stores: 3
Número de registros en Staffs: 10
Número de registros en Orders: 1615
Número de registros en Order_Items: 4722
Número de registros en Stocks: 939


### Misión 5: Consultas


Creamos una función para obtener resultados de consultas. 


In [63]:
def ejecutar(consulta):
    cursor.execute(consulta)
    resultado = cursor.fetchall()
    return resultado


a) Liste todos los productos junto con su nombre de categoría y marca.


In [64]:
consulta = """
SELECT P.product_name, C.category_name, B.brand_name
FROM Products P
JOIN Categories C ON P.category_id = C.category_id
JOIN Brands B ON P.brand_id = B.brand_id
ORDER BY P.product_name
"""

resultado = ejecutar(consulta)
print("Producto | Categoría | Marca") # Esto se lo pedí al chat para que se vea mejor la separación entre cada columna. 
for producto, categoria, marca in resultado[:10]:  # Mostramos los primeros 10 resultados.
    print(f'{producto} | {categoria} | {marca}')
print(f"Total de productos: {len(resultado)}")


Producto | Categoría | Marca
Electra Amsterdam Fashion 3i Ladies' - 2017/2018 | Cruisers Bicycles | Electra
Electra Amsterdam Fashion 7i Ladies' - 2017 | Cruisers Bicycles | Electra
Electra Amsterdam Original 3i - 2015/2017 | Cruisers Bicycles | Electra
Electra Amsterdam Original 3i Ladies' - 2017 | Cruisers Bicycles | Electra
Electra Amsterdam Royal 8i - 2017/2018 | Cruisers Bicycles | Electra
Electra Amsterdam Royal 8i Ladies - 2018 | Cruisers Bicycles | Electra
Electra Cruiser 1 (24-Inch) - 2016 | Cruisers Bicycles | Electra
Electra Cruiser 1 (24-Inch) - 2016 | Children Bicycles | Electra
Electra Cruiser 1 - 2016/2017/2018 | Cruisers Bicycles | Electra
Electra Cruiser 1 Ladies' - 2018 | Cruisers Bicycles | Electra
Total de productos: 321


In [65]:
# Validamo con Python, codigo solicitado al chat. 
productos_validacion = products_df.merge(categories_df, on='category_id', how='left').merge(brands_df, on='brand_id', how='left') # Acá lo que se hace es unir las tablas de categorías y marcas con la tabla de productos. 
productos_validacion = productos_validacion[['product_name', 'category_name', 'brand_name']].sort_values('product_name') # Acá lo que se hace es ordenar los productos por nombre. 
print(f"Total de productos luego de la validación: {len(productos_validacion)}")
display(productos_validacion.head(10))


Total de productos luego de la validación: 321


,product_name,category_name,brand_name
256,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,Cruisers Bicycles,Electra
80,Electra Amsterdam Fashion 7i Ladies' - 2017,Cruisers Bicycles,Electra
69,Electra Amsterdam Original 3i - 2015/2017,Cruisers Bicycles,Electra
81,Electra Amsterdam Original 3i Ladies' - 2017,Cruisers Bicycles,Electra
257,Electra Amsterdam Royal 8i - 2017/2018,Cruisers Bicycles,Electra
258,Electra Amsterdam Royal 8i Ladies - 2018,Cruisers Bicycles,Electra
12,Electra Cruiser 1 (24-Inch) - 2016,Cruisers Bicycles,Electra
20,Electra Cruiser 1 (24-Inch) - 2016,Children Bicycles,Electra
212,Electra Cruiser 1 - 2016/2017/2018,Cruisers Bicycles,Electra
219,Electra Cruiser 1 Ladies' - 2018,Cruisers Bicycles,Electra


b) Muestre cuántos clientes están registrados por ciudad.


In [66]:
consulta = """
SELECT city, COUNT(*) as cantidad
FROM Customers
GROUP BY city
ORDER BY cantidad DESC
"""

resultado = ejecutar(consulta)
print("Ciudad | Cantidad de clientes")
for ciudad, cantidad in resultado:
    print(f'{ciudad}: {cantidad}')


Ciudad | Cantidad de clientes
Mount Vernon: 20
Scarsdale: 17
Ballston Spa: 17
Canandaigua: 14
Ossining: 13
Longview: 13
Floral Park: 13
Webster: 12
Sunnyside: 12
Smithtown: 12
San Angelo: 12
Ronkonkoma: 12
Richmond Hill: 12
Merrick: 12
Howard Beach: 12
Canyon Country: 12
Astoria: 12
Upland: 11
South El Monte: 11
Orchard Park: 11
Monsey: 11
Monroe: 11
Kingston: 11
Jamaica: 11
Central Islip: 11
Bay Shore: 11
Apple Valley: 11
Anaheim: 11
Woodside: 10
Troy: 10
Spring Valley: 10
Shirley: 10
San Lorenzo: 10
Port Washington: 10
Palos Verdes Peninsula: 10
Oxnard: 10
New Windsor: 10
Maspeth: 10
Long Beach: 10
Liverpool: 10
Franklin Square: 10
Euless: 10
Elmhurst: 10
East Elmhurst: 10
Campbell: 10
Uniondale: 9
San Carlos: 9
Rochester: 9
Pittsford: 9
North Tonawanda: 9
Newburgh: 9
Lindenhurst: 9
Lancaster: 9
Houston: 9
East Northport: 9
Duarte: 9
Buffalo: 9
Brooklyn: 9
Bronx: 9
Brentwood: 9
Amityville: 9
Woodhaven: 8
Whitestone: 8
Utica: 8
South Ozone Park: 8
Schenectady: 8
Saratoga Springs: 8
Sa

In [67]:
# Validación con Python, codigo también solicitado al chat. 
clientes_por_ciudad = customers_df.groupby('city').size().reset_index(name='cantidad').sort_values('cantidad', ascending=False) # Básicamente lo que se hace es agrupar los clientes por ciudad y contar la cantidad de clientes en cada ciudad. 
print("Clientes por ciudad luego de la validación:")
display(clientes_por_ciudad.head(10))


Clientes por ciudad luego de la validación:


,city,cantidad
102,Mount Vernon,20
12,Ballston Spa,17
159,Scarsdale,17
23,Canandaigua,14
118,Ossining,13
90,Longview,13
51,Floral Park,13
24,Canyon Country,12
98,Merrick,12
74,Howard Beach,12


c) Calcule el precio promedio de los productos vendidos.


In [68]:
consulta = """
SELECT AVG(list_price) as precio_promedio
FROM Order_Items
"""

resultado = ejecutar(consulta)
precio_promedio = resultado[0][0]
print(f"Precio promedio de productos vendidos: ${precio_promedio:.2f}")


Precio promedio de productos vendidos: $1212.71


In [69]:
# Validación con Python
precio_promedio_python = order_items_df['list_price'].mean()
print(f"Precio promedio luego de la validación: ${precio_promedio_python:.2f}")


Precio promedio luego de la validación: $1212.71


d) Obtenga las 5 tiendas con más órdenes realizadas.


In [70]:
consulta = """
SELECT S.store_name, COUNT(O.order_id) as cantidad_ordenes
FROM Stores S
JOIN Orders O ON S.store_id = O.store_id
GROUP BY S.store_id
ORDER BY cantidad_ordenes DESC
LIMIT 5
"""

resultado = ejecutar(consulta)
print("Tienda | Cantidad de órdenes")
for tienda, cantidad in resultado:
    print(f'{tienda}: {cantidad}')


Tienda | Cantidad de órdenes
Baldwin Bikes: 1093
Santa Cruz Bikes: 348
Rowlett Bikes: 174


In [71]:
# Validación con Python
ordenes_por_tienda = orders_df.groupby('store_id').size().reset_index(name='cantidad') # Mismo cpodigo de antes pero no los ordenamos de mayor a menor como si se hacñia antes. 
ordenes_por_tienda = ordenes_por_tienda.merge(stores_df[['store_id', 'store_name']], on='store_id')
ordenes_por_tienda = ordenes_por_tienda.sort_values('cantidad', ascending=False).head(5)
print("Top 5 tiendas luego de la validación:")
display(ordenes_por_tienda[['store_name', 'cantidad']])


Top 5 tiendas luego de la validación:


,store_name,cantidad
1,Baldwin Bikes,1093
0,Santa Cruz Bikes,348
2,Rowlett Bikes,174


e) Liste los empleados que han vendido productos de al menos 5 categorías diferentes.


In [72]:
consulta = """
SELECT St.first_name || ' ' || St.last_name as nombre_empleado, COUNT(DISTINCT P.category_id) as categorias_diferentes
FROM Staffs St
JOIN Orders O ON St.staff_id = O.staff_id
JOIN Order_Items OI ON O.order_id = OI.order_id
JOIN Products P ON OI.product_id = P.product_id
GROUP BY St.staff_id
HAVING COUNT(DISTINCT P.category_id) >= 5
ORDER BY categorias_diferentes DESC
"""

resultado = ejecutar(consulta)
print("Empleado | Categorías diferentes\n")
for empleado, categorias in resultado:
    print(f'{empleado}: {categorias}')


Empleado | Categorías diferentes

Layla Terrell: 7
Kali Vargas: 7
Venita Daniel: 7
Marcelene Boyer: 7
Genna Serrano: 7
Mireya Copeland: 7


In [73]:
# Validación con Python
orders_df = pd.read_csv('E3_datos/orders.csv')
order_items_df = pd.read_csv('E3_datos/order_items.csv')
products_df = pd.read_csv('E3_datos/products.csv')
staffs_df = pd.read_csv('E3_datos/staffs.csv')

empleados_ordenes = orders_df[['order_id', 'staff_id']].merge(order_items_df[['order_id', 'product_id']], on='order_id')
empleados_productos = empleados_ordenes.merge(products_df[['product_id', 'category_id']], on='product_id')
empleados_categorias = empleados_productos.groupby('staff_id')['category_id'].nunique().reset_index(name='categorias') # Acá lo que se hace es agrupar los empleados por staff_id y contar la cantidad de categorías diferentes. 
empleados_categorias = empleados_categorias[empleados_categorias['categorias'] >= 5]
empleados_categorias = empleados_categorias.merge(staffs_df[['staff_id', 'first_name', 'last_name']], on='staff_id')
empleados_categorias['nombre'] = empleados_categorias['first_name'] + ' ' + empleados_categorias['last_name']
print("Empleados con al menos 5 categorías luego de la validación:")
display(empleados_categorias[['nombre', 'categorias']].sort_values('categorias', ascending=False))


Empleados con al menos 5 categorías luego de la validación:


,nombre,categorias
0,Mireya Copeland,7
1,Genna Serrano,7
2,Marcelene Boyer,7
3,Venita Daniel,7
4,Kali Vargas,7
5,Layla Terrell,7


f) Calcule el total vendido por tienda y mes.


In [74]:
consulta = """
SELECT S.store_name, 
       strftime('%Y-%m', O.order_date) as mes,
       SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as total_vendido
FROM Stores S
JOIN Orders O ON S.store_id = O.store_id
JOIN Order_Items OI ON O.order_id = OI.order_id
GROUP BY S.store_id, mes
ORDER BY S.store_name, mes
"""

resultado = ejecutar(consulta)
print("Tienda | Mes | Total vendido\n")
for tienda, mes, total in resultado[:20]:  # Mostramos los primeros 20
    print(f'{tienda} | {mes} | ${total:.2f}')

Tienda | Mes | Total vendido

Baldwin Bikes | 2016-01 | $132894.30
Baldwin Bikes | 2016-02 | $102201.91
Baldwin Bikes | 2016-03 | $110338.79
Baldwin Bikes | 2016-04 | $120491.98
Baldwin Bikes | 2016-05 | $150679.65
Baldwin Bikes | 2016-06 | $139021.39
Baldwin Bikes | 2016-07 | $167320.87
Baldwin Bikes | 2016-08 | $142994.29
Baldwin Bikes | 2016-09 | $164368.65
Baldwin Bikes | 2016-10 | $129251.73
Baldwin Bikes | 2016-11 | $95705.80
Baldwin Bikes | 2016-12 | $135329.51
Baldwin Bikes | 2017-01 | $217406.65
Baldwin Bikes | 2017-02 | $197814.73
Baldwin Bikes | 2017-03 | $219582.69
Baldwin Bikes | 2017-04 | $161639.93
Baldwin Bikes | 2017-05 | $201408.00
Baldwin Bikes | 2017-06 | $250491.80
Baldwin Bikes | 2017-07 | $171303.28
Baldwin Bikes | 2017-08 | $193430.86


In [75]:
# Validación con Python
ordenes_items = orders_df[['order_id', 'store_id', 'order_date']].merge(
    order_items_df[['order_id', 'quantity', 'list_price', 'discount']], on='order_id')
ordenes_items['total'] = ordenes_items['quantity'] * ordenes_items['list_price'] * (1 - ordenes_items['discount'])
ordenes_items['mes'] = pd.to_datetime(ordenes_items['order_date']).dt.to_period('M').astype(str)
ventas_por_tienda_mes = ordenes_items.groupby(['store_id', 'mes'])['total'].sum().reset_index()
ventas_por_tienda_mes = ventas_por_tienda_mes.merge(stores_df[['store_id', 'store_name']], on='store_id')
print("Validación Python - Total vendido por tienda y mes:")
display(ventas_por_tienda_mes.head(20))


Validación Python - Total vendido por tienda y mes:


,store_id,mes,total,store_name
0,1,2016-01,71760.3064,Santa Cruz Bikes
1,1,2016-02,35255.4513,Santa Cruz Bikes
2,1,2016-03,46817.6251,Santa Cruz Bikes
3,1,2016-04,34230.6401,Santa Cruz Bikes
4,1,2016-05,28095.2399,Santa Cruz Bikes
5,1,2016-06,41554.1683,Santa Cruz Bikes
6,1,2016-07,21258.8857,Santa Cruz Bikes
7,1,2016-08,44803.6003,Santa Cruz Bikes
8,1,2016-09,65159.6319,Santa Cruz Bikes
9,1,2016-10,69175.4980,Santa Cruz Bikes


g) Encuentre productos que han generado más ingresos que el promedio de su categoría.


In [76]:
consulta = """
SELECT P.product_name, C.category_name, 
       SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as ingresos_producto,
       AVG(ingresos_categoria.ingresos) as promedio_categoria
FROM Products P
JOIN Categories C ON P.category_id = C.category_id
JOIN Order_Items OI ON P.product_id = OI.product_id
JOIN (
    SELECT P2.category_id, 
           SUM(OI2.quantity * OI2.list_price * (1 - OI2.discount)) as ingresos
    FROM Products P2
    JOIN Order_Items OI2 ON P2.product_id = OI2.product_id
    GROUP BY P2.product_id
) ingresos_categoria ON P.category_id = ingresos_categoria.category_id
GROUP BY P.product_id
HAVING SUM(OI.quantity * OI.list_price * (1 - OI.discount)) > 
       (SELECT AVG(ingresos_cat.ingresos)
        FROM (
            SELECT P3.product_id,
                   SUM(OI3.quantity * OI3.list_price * (1 - OI3.discount)) as ingresos
            FROM Products P3
            JOIN Order_Items OI3 ON P3.product_id = OI3.product_id
            WHERE P3.category_id = P.category_id
            GROUP BY P3.product_id
        ) ingresos_cat)
ORDER BY ingresos_producto DESC
"""

resultado = ejecutar(consulta)
print("Producto | Categoría | Ingresos producto | Promedio categoría\n")
for producto, categoria, ingresos, promedio in resultado[:10]:
    print(f'{producto} | {categoria} | ${ingresos:.2f} | ${promedio:.2f}')


Producto | Categoría | Ingresos producto | Promedio categoría

Trek Slash 8 27.5 - 2016 | Mountain Bikes | $31666840.83 | $47632.97
Trek Fuel EX 8 29 - 2016 | Mountain Bikes | $21002945.58 | $47632.97
Trek Remedy 29 Carbon Frameset - 2016 | Mountain Bikes | $11592709.60 | $47632.97
Trek Domane SLR 6 Disc - 2017 | Road Bikes | $11213984.61 | $31416.95
Trek Madone 9.2 - 2017 | Road Bikes | $9322681.35 | $31416.95
Trek Silque SLR 8 Women's - 2017 | Road Bikes | $9249810.77 | $31416.95
Trek Fuel EX 9.8 27.5 Plus - 2017 | Mountain Bikes | $9066003.89 | $47632.97
Trek Conduit+ - 2016 | Electric Bikes | $8952720.16 | $39855.86
Trek Silque SLR 7 Women's - 2017 | Road Bikes | $8643225.59 | $31416.95
Heller Shagamaw Frame - 2016 | Mountain Bikes | $8616170.48 | $47632.97


In [77]:
# Validación con Python
items_ingresos = order_items_df.copy()
items_ingresos['ingresos'] = items_ingresos['quantity'] * items_ingresos['list_price'] * (1 - items_ingresos['discount'])
ingresos_por_producto = items_ingresos.groupby('product_id')['ingresos'].sum().reset_index()
ingresos_por_producto = ingresos_por_producto.merge(products_df[['product_id', 'product_name', 'category_id']], on='product_id')
ingresos_por_producto = ingresos_por_producto.merge(categories_df[['category_id', 'category_name']], on='category_id')
promedio_por_categoria = ingresos_por_producto.groupby('category_id')['ingresos'].mean().reset_index(name='promedio_categoria')
productos_sobre_promedio = ingresos_por_producto.merge(promedio_por_categoria, on='category_id')
productos_sobre_promedio = productos_sobre_promedio[productos_sobre_promedio['ingresos'] > productos_sobre_promedio['promedio_categoria']]
print("Validación Python - Productos sobre el promedio de su categoría:")
display(productos_sobre_promedio[['product_name', 'category_name', 'ingresos', 'promedio_categoria']].head(10))


Validación Python - Productos sobre el promedio de su categoría:


,product_name,category_name,ingresos,promedio_categoria
0,Ritchey Timberwolf Frameset - 2016,Mountain Bikes,78898.9480,47632.974275
1,Surly Wednesday Frameset - 2016,Mountain Bikes,112288.8771,47632.974275
2,Trek Fuel EX 8 29 - 2016,Mountain Bikes,368472.7294,47632.974275
3,Heller Shagamaw Frame - 2016,Mountain Bikes,151160.8857,47632.974275
4,Surly Ice Cream Truck Frameset - 2016,Mountain Bikes,70371.6027,47632.974275
5,Trek Slash 8 27.5 - 2016,Mountain Bikes,555558.6111,47632.974275
6,Trek Remedy 29 Carbon Frameset - 2016,Mountain Bikes,203380.8701,47632.974275
7,Trek Conduit+ - 2016,Electric Bikes,389248.7025,39855.860000
8,Surly Straggler - 2016,Cyclocross Bicycles,203507.6200,71101.183590
9,Surly Straggler 650b - 2016,Cyclocross Bicycles,226765.5510,71101.183590


h) Liste los 10 clientes que m´as gastaron en productos que nunca se han vuelto a vender despu´es.

In [78]:
consulta = """
SELECT C.first_name || ' ' || C.last_name as nombre_cliente,
       SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as total_gastado
FROM Customers C
JOIN Orders O ON C.customer_id = O.customer_id
JOIN Order_Items OI ON O.order_id = OI.order_id
WHERE OI.product_id IN (
    SELECT product_id
    FROM Order_Items
    GROUP BY product_id
    HAVING COUNT(DISTINCT order_id) = 1
)
GROUP BY C.customer_id
ORDER BY total_gastado DESC
LIMIT 10
"""

resultado = ejecutar(consulta)
print("Cliente | Total gastado\n")
for cliente, total in resultado:
    print(f'{cliente}: ${total:.2f}')


Cliente | Total gastado

Hedwig Paul: $10449.98
Santa Larson: $7869.97
Jana Thomas: $6174.99
Desmond Rose: $4987.48
Louanne Martin: $4835.98
Earlean Pena: $4749.67
Jamaal Morrison: $3254.99
Kara Higgins: $3092.98
Lolita Mosley: $2975.99
Lyndsey Bean: $2943.10


In [79]:
# Validación con Python
productos_una_venta = order_items_df.groupby('product_id')['order_id'].nunique()
productos_una_venta = productos_una_venta[productos_una_venta == 1].index.tolist()
items_productos_unicos = order_items_df[order_items_df['product_id'].isin(productos_una_venta)]
items_productos_unicos['total'] = items_productos_unicos['quantity'] * items_productos_unicos['list_price'] * (1 - items_productos_unicos['discount'])
items_con_ordenes = items_productos_unicos.merge(orders_df[['order_id', 'customer_id']], on='order_id')
gasto_por_cliente = items_con_ordenes.groupby('customer_id')['total'].sum().reset_index(name='total_gastado')
gasto_por_cliente = gasto_por_cliente.merge(customers_df[['customer_id', 'first_name', 'last_name']], on='customer_id')
gasto_por_cliente['nombre'] = gasto_por_cliente['first_name'] + ' ' + gasto_por_cliente['last_name']
print("Validación Python - Top 10 clientes:")
display(gasto_por_cliente[['nombre', 'total_gastado']].sort_values('total_gastado', ascending=False).head(10))


Validación Python - Top 10 clientes:


/var/folders/z5/wvf1lf5579ld90s3v_k6rdt00000gn/T/ipykernel_8009/1443571347.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_productos_unicos['total'] = items_productos_unicos['quantity'] * items_productos_unicos['list_price'] * (1 - items_productos_unicos['discount'])


,nombre,total_gastado
19,Hedwig Paul,10449.9810
31,Santa Larson,7869.9725
27,Jana Thomas,6174.9905
30,Desmond Rose,4987.4810
16,Louanne Martin,4835.9814
32,Earlean Pena,4749.6747
35,Jamaal Morrison,3254.9907
26,Kara Higgins,3092.9817
13,Lolita Mosley,2975.9907
2,Lyndsey Bean,2943.1000


i) Encuentre los productos con m´as unidades en stock que ventas totales acumuladas.

In [80]:
consulta = """
SELECT P.product_name,
       SUM(S.quantity) as stock_total,
       COALESCE(SUM(OI.quantity), 0) as ventas_totales
FROM Products P
JOIN Stocks S ON P.product_id = S.product_id
LEFT JOIN Order_Items OI ON P.product_id = OI.product_id
GROUP BY P.product_id
HAVING SUM(S.quantity) > COALESCE(SUM(OI.quantity), 0)
ORDER BY stock_total DESC
LIMIT 10
"""

resultado = ejecutar(consulta)
print("Producto | Stock total | Ventas totales\n")
for producto, stock, ventas in resultado:
    print(f'{producto} | {stock} | {ventas}')


Producto | Stock total | Ventas totales

Surly Straggler 650b - 2016 | 5723 | 453
Electra Townie Original 7D EQ - Women's - 2016 | 5460 | 375
Trek Conduit+ - 2016 | 5151 | 435
Electra Townie Original 21D - 2016 | 4992 | 459
Surly Straggler - 2016 | 4753 | 441
Electra Girl's Hawaii 1 (16-inch) - 2015/2016 | 4606 | 435
Electra Cruiser 1 (24-Inch) - 2016 | 4320 | 417
Electra Townie Original 7D - 2015/2016 | 4214 | 444
Surly Ice Cream Truck Frameset - 2016 | 4180 | 501
Electra Townie Original 7D EQ - 2016 | 4085 | 468


In [81]:
# Validación con Python
stock_total = stocks_df.groupby('product_id')['quantity'].sum().reset_index(name='stock_total')
ventas_totales = order_items_df.groupby('product_id')['quantity'].sum().reset_index(name='ventas_totales')
productos_stock_vs_ventas = stock_total.merge(ventas_totales, on='product_id', how='left')
productos_stock_vs_ventas['ventas_totales'] = productos_stock_vs_ventas['ventas_totales'].fillna(0)
productos_stock_vs_ventas = productos_stock_vs_ventas[productos_stock_vs_ventas['stock_total'] > productos_stock_vs_ventas['ventas_totales']]
productos_stock_vs_ventas = productos_stock_vs_ventas.merge(products_df[['product_id', 'product_name']], on='product_id')
print("Validación Python - Productos con más stock que ventas:")
display(productos_stock_vs_ventas[['product_name', 'stock_total', 'ventas_totales']].sort_values('stock_total', ascending=False).head(10))


Validación Python - Productos con más stock que ventas:


,product_name,stock_total,ventas_totales
138,Trek XM700+ Lowstep - 2018,86,7.0
26,Electra Townie Original 7D - 2017,82,27.0
60,Sun Bicycles Cruz 7 - Women's - 2017,79,35.0
146,Trek Verve+ - 2018,79,5.0
25,Trek Powerfly 8 FS Plus - 2017,78,41.0
132,Trek Domane SL 5 Disc - 2018,77,5.0
116,Trek Domane AL 2 Women's - 2018,77,5.0
169,Electra Moto 3i - 2018,75,5.0
92,Trek Fuel EX 8 29 XT - 2018,75,10.0
202,Electra Townie Commute Go! Ladies' - 2018,75,11.0


j) Calcule la variación mensual del total vendido por tienda, expresada como porcentaje respecto al mes anterior.


In [82]:
consulta = """
WITH VentasMensuales AS (
    SELECT S.store_id,
           S.store_name,
           strftime('%Y-%m', O.order_date) as mes,
           SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as total_vendido
    FROM Stores S
    JOIN Orders O ON S.store_id = O.store_id
    JOIN Order_Items OI ON O.order_id = OI.order_id
    GROUP BY S.store_id, mes
)
SELECT vm1.store_name,
       vm1.mes as mes_actual,
       vm1.total_vendido as ventas_actuales,
       vm2.total_vendido as ventas_anterior,
       CASE 
           WHEN vm2.total_vendido > 0 
           THEN ROUND(((vm1.total_vendido - vm2.total_vendido) / vm2.total_vendido) * 100, 2)
           ELSE NULL
       END as variacion_porcentual
FROM VentasMensuales vm1
LEFT JOIN VentasMensuales vm2 ON vm1.store_id = vm2.store_id 
    AND vm2.mes = date(vm1.mes || '-01', '-1 month')
WHERE vm2.mes IS NOT NULL
ORDER BY vm1.store_name, vm1.mes
"""

resultado = ejecutar(consulta)
print("Tienda | Mes actual | Ventas actuales | Ventas anterior | Variación %\n")
for tienda, mes, ventas_act, ventas_ant, variacion in resultado[:20]:
    print(f'{tienda} | {mes} | ${ventas_act:.2f} | ${ventas_ant:.2f} | {variacion}%')


Tienda | Mes actual | Ventas actuales | Ventas anterior | Variación %



In [83]:
# Validación con Python
ordenes_items = orders_df[['order_id', 'store_id', 'order_date']].merge(
    order_items_df[['order_id', 'quantity', 'list_price', 'discount']], on='order_id')
ordenes_items['total'] = ordenes_items['quantity'] * ordenes_items['list_price'] * (1 - ordenes_items['discount'])
ordenes_items['mes'] = pd.to_datetime(ordenes_items['order_date']).dt.to_period('M').astype(str)
ventas_mensuales = ordenes_items.groupby(['store_id', 'mes'])['total'].sum().reset_index()
ventas_mensuales = ventas_mensuales.merge(stores_df[['store_id', 'store_name']], on='store_id')
ventas_mensuales = ventas_mensuales.sort_values(['store_id', 'mes'])
ventas_mensuales['ventas_anterior'] = ventas_mensuales.groupby('store_id')['total'].shift(1)
ventas_mensuales['variacion_porcentual'] = ((ventas_mensuales['total'] - ventas_mensuales['ventas_anterior']) / ventas_mensuales['ventas_anterior']) * 100
ventas_mensuales = ventas_mensuales.dropna()
print("Validación Python - Variación mensual:")
display(ventas_mensuales[['store_name', 'mes', 'total', 'ventas_anterior', 'variacion_porcentual']].head(20))


Validación Python - Variación mensual:


,store_name,mes,total,ventas_anterior,variacion_porcentual
1,Santa Cruz Bikes,2016-02,35255.4513,71760.3064,-50.870540
2,Santa Cruz Bikes,2016-03,46817.6251,35255.4513,32.795421
3,Santa Cruz Bikes,2016-04,34230.6401,46817.6251,-26.885142
4,Santa Cruz Bikes,2016-05,28095.2399,34230.6401,-17.923709
5,Santa Cruz Bikes,2016-06,41554.1683,28095.2399,47.904657
6,Santa Cruz Bikes,2016-07,21258.8857,41554.1683,-48.840546
7,Santa Cruz Bikes,2016-08,44803.6003,21258.8857,110.752346
8,Santa Cruz Bikes,2016-09,65159.6319,44803.6003,45.433919
9,Santa Cruz Bikes,2016-10,69175.4980,65159.6319,6.163120
10,Santa Cruz Bikes,2016-11,55771.3843,69175.4980,-19.376967


k) Liste productos cuyo precio promedio de venta ha disminuido mes a mes durante al menos tres meses consecutivos.


In [84]:
consulta = """
WITH PreciosMensuales AS (
    SELECT P.product_id,
           P.product_name,
           strftime('%Y-%m', O.order_date) as mes,
           AVG(OI.list_price) as precio_promedio
    FROM Products P
    JOIN Order_Items OI ON P.product_id = OI.product_id
    JOIN Orders O ON OI.order_id = O.order_id
    GROUP BY P.product_id, mes
),
PreciosConAnterior AS (
    SELECT product_id,
           product_name,
           mes,
           precio_promedio,
           LAG(precio_promedio) OVER (PARTITION BY product_id ORDER BY mes) as precio_anterior
    FROM PreciosMensuales
),
ProductosConDisminucion AS (
    SELECT product_id,
           product_name,
           mes,
           precio_promedio,
           precio_anterior,
           CASE WHEN precio_anterior IS NOT NULL AND precio_promedio < precio_anterior THEN 1 ELSE 0 END as disminuyo
    FROM PreciosConAnterior
)
SELECT DISTINCT product_name
FROM (
    SELECT product_id,
           product_name,
           mes,
           disminuyo,
           SUM(disminuyo) OVER (PARTITION BY product_id ORDER BY mes ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as consecutivos
    FROM ProductosConDisminucion
) 
WHERE consecutivos >= 3
ORDER BY product_name
"""

resultado = ejecutar(consulta)
print("Productos con precio promedio disminuyendo 3+ meses consecutivos:\n")
for producto in resultado:
    print(producto[0])


Productos con precio promedio disminuyendo 3+ meses consecutivos:



In [85]:
# Validación con Python
items_con_fecha = order_items_df[['order_id', 'product_id', 'list_price']].merge(
    orders_df[['order_id', 'order_date']], on='order_id')
items_con_fecha['mes'] = pd.to_datetime(items_con_fecha['order_date']).dt.to_period('M').astype(str)
precios_mensuales = items_con_fecha.groupby(['product_id', 'mes'])['list_price'].mean().reset_index(name='precio_promedio')
precios_mensuales = precios_mensuales.sort_values(['product_id', 'mes'])
precios_mensuales['precio_anterior'] = precios_mensuales.groupby('product_id')['precio_promedio'].shift(1)
precios_mensuales['disminuyo'] = (precios_mensuales['precio_promedio'] < precios_mensuales['precio_anterior']).astype(int)
precios_mensuales['consecutivos'] = precios_mensuales.groupby('product_id')['disminuyo'].transform(
    lambda x: x.rolling(window=3, min_periods=3).sum())
productos_disminucion = precios_mensuales[precios_mensuales['consecutivos'] >= 3]['product_id'].unique()
productos_disminucion_df = products_df[products_df['product_id'].isin(productos_disminucion)][['product_name']]
print("Validación Python - Productos con disminución consecutiva:")
display(productos_disminucion_df.head(10))


Validación Python - Productos con disminución consecutiva:


,product_name


l) Liste todos los clientes que han comprado productos de todas las categorías existentes.


In [86]:
consulta = """
SELECT C.first_name || ' ' || C.last_name as nombre_cliente
FROM Customers C
WHERE (
    SELECT COUNT(DISTINCT P.category_id)
    FROM Orders O
    JOIN Order_Items OI ON O.order_id = OI.order_id
    JOIN Products P ON OI.product_id = P.product_id
    WHERE O.customer_id = C.customer_id
) = (SELECT COUNT(*) FROM Categories)
"""

resultado = ejecutar(consulta)
print("Clientes que compraron de todas las categorías:")
for cliente in resultado:
    print(cliente[0])


Clientes que compraron de todas las categorías:
Genoveva Baldwin


In [87]:
# Validación con Python
total_categorias = len(categories_df)
items_con_categoria = order_items_df[['order_id', 'product_id']].merge(
    products_df[['product_id', 'category_id']], on='product_id')
ordenes_con_categoria = items_con_categoria.merge(orders_df[['order_id', 'customer_id']], on='order_id')
categorias_por_cliente = ordenes_con_categoria.groupby('customer_id')['category_id'].nunique().reset_index(name='categorias_compradas')
clientes_todas_categorias = categorias_por_cliente[categorias_por_cliente['categorias_compradas'] == total_categorias]
clientes_todas_categorias = clientes_todas_categorias.merge(customers_df[['customer_id', 'first_name', 'last_name']], on='customer_id')
clientes_todas_categorias['nombre'] = clientes_todas_categorias['first_name'] + ' ' + clientes_todas_categorias['last_name']
print("Validación Python - Clientes con todas las categorías:")
display(clientes_todas_categorias[['nombre']])


Validación Python - Clientes con todas las categorías:


,nombre
0,Genoveva Baldwin


m) Calcule la secuencia más larga de días consecutivos con ventas en cada tienda.


In [88]:
consulta = """
WITH FechasVentas AS (
    SELECT DISTINCT S.store_id,
           S.store_name,
           DATE(O.order_date) as fecha_venta
    FROM Stores S
    JOIN Orders O ON S.store_id = O.store_id
),
Secuencias AS (
    SELECT store_id,
           store_name,
           fecha_venta,
           ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY fecha_venta) as rn,
           DATE(fecha_venta, '-' || (ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY fecha_venta) - 1) || ' days') as grupo
    FROM FechasVentas
),
ConteoConsecutivos AS (
    SELECT store_id,
           store_name,
           grupo,
           COUNT(*) as dias_consecutivos,
           MIN(fecha_venta) as inicio,
           MAX(fecha_venta) as fin
    FROM Secuencias
    GROUP BY store_id, grupo
)
SELECT store_name,
       MAX(dias_consecutivos) as max_dias_consecutivos
FROM ConteoConsecutivos
GROUP BY store_id
ORDER BY max_dias_consecutivos DESC
"""

resultado = ejecutar(consulta)
print("Tienda | Máximo días consecutivos con ventas\n")
for tienda, dias in resultado:
    print(f'{tienda}: {dias} días')


Tienda | Máximo días consecutivos con ventas

Baldwin Bikes: 15 días
Santa Cruz Bikes: 11 días
Rowlett Bikes: 4 días


In [89]:
# Validación con Python
fechas_ventas = orders_df[['store_id', 'order_date']].copy()
fechas_ventas['fecha'] = pd.to_datetime(fechas_ventas['order_date']).dt.date
fechas_ventas = fechas_ventas.drop_duplicates(['store_id', 'fecha']).sort_values(['store_id', 'fecha'])

def max_consecutivos(group):
    fechas = sorted(group['fecha'].unique())
    if len(fechas) == 0:
        return 0
    max_seq = 1
    current_seq = 1
    for i in range(1, len(fechas)):
        if (fechas[i] - fechas[i-1]).days == 1:
            current_seq += 1
            max_seq = max(max_seq, current_seq)
        else:
            current_seq = 1
    return max_seq

max_consecutivos_por_tienda = fechas_ventas.groupby('store_id').apply(max_consecutivos).reset_index(name='max_dias')
max_consecutivos_por_tienda = max_consecutivos_por_tienda.merge(stores_df[['store_id', 'store_name']], on='store_id')
print("Validación Python - Máximo días consecutivos:")
display(max_consecutivos_por_tienda[['store_name', 'max_dias']].sort_values('max_dias', ascending=False))


Validación Python - Máximo días consecutivos:


/var/folders/z5/wvf1lf5579ld90s3v_k6rdt00000gn/T/ipykernel_8009/249132639.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_consecutivos_por_tienda = fechas_ventas.groupby('store_id').apply(max_consecutivos).reset_index(name='max_dias')


,store_name,max_dias
1,Baldwin Bikes,15
0,Santa Cruz Bikes,11
2,Rowlett Bikes,4


n) Determine los productos que fueron top 1 en ingresos mensuales durante al menos 6 meses no consecutivos.


In [90]:
consulta = """
WITH IngresosMensuales AS (
    SELECT P.product_id,
           P.product_name,
           strftime('%Y-%m', O.order_date) as mes,
           SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as ingresos
    FROM Products P
    JOIN Order_Items OI ON P.product_id = OI.product_id
    JOIN Orders O ON OI.order_id = O.order_id
    GROUP BY P.product_id, mes
),
TopPorMes AS (
    SELECT product_id,
           product_name,
           mes,
           ingresos,
           RANK() OVER (PARTITION BY mes ORDER BY ingresos DESC) as ranking
    FROM IngresosMensuales
),
Top1Productos AS (
    SELECT product_id,
           product_name,
           COUNT(DISTINCT mes) as meses_top1
    FROM TopPorMes
    WHERE ranking = 1
    GROUP BY product_id
)
SELECT product_name,
       meses_top1
FROM Top1Productos
WHERE meses_top1 >= 6
ORDER BY meses_top1 DESC
"""

resultado = ejecutar(consulta)
print("Producto | Meses como top 1\n")
for producto, meses in resultado:
    print(f'{producto}: {meses} meses')


Producto | Meses como top 1

Trek Slash 8 27.5 - 2016: 9 meses


In [91]:
# Validación con Python
items_ingresos = order_items_df[['order_id', 'product_id', 'quantity', 'list_price', 'discount']].copy()
items_ingresos['ingresos'] = items_ingresos['quantity'] * items_ingresos['list_price'] * (1 - items_ingresos['discount'])
items_con_fecha = items_ingresos.merge(orders_df[['order_id', 'order_date']], on='order_id')
items_con_fecha['mes'] = pd.to_datetime(items_con_fecha['order_date']).dt.to_period('M').astype(str)
ingresos_mensuales = items_con_fecha.groupby(['product_id', 'mes'])['ingresos'].sum().reset_index()
ingresos_mensuales = ingresos_mensuales.merge(products_df[['product_id', 'product_name']], on='product_id')
top_por_mes = ingresos_mensuales.sort_values(['mes', 'ingresos'], ascending=[True, False])
top_por_mes['ranking'] = top_por_mes.groupby('mes')['ingresos'].rank(method='first', ascending=False)
top1_productos = top_por_mes[top_por_mes['ranking'] == 1]
meses_top1 = top1_productos.groupby('product_id')['mes'].nunique().reset_index(name='meses_top1')
meses_top1 = meses_top1[meses_top1['meses_top1'] >= 6]
meses_top1 = meses_top1.merge(products_df[['product_id', 'product_name']], on='product_id')
print("Validación Python - Productos top 1 en 6+ meses:")
display(meses_top1[['product_name', 'meses_top1']].sort_values('meses_top1', ascending=False))


Validación Python - Productos top 1 en 6+ meses:


,product_name,meses_top1
0,Trek Slash 8 27.5 - 2016,9


o) Encuentre el subconjunto mínimo de productos que representa al menos el 80% del ingreso total.


In [92]:
consulta = """
WITH IngresosPorProducto AS (
    SELECT P.product_id,
           P.product_name,
           SUM(OI.quantity * OI.list_price * (1 - OI.discount)) as ingresos
    FROM Products P
    JOIN Order_Items OI ON P.product_id = OI.product_id
    GROUP BY P.product_id
),
IngresosAcumulados AS (
    SELECT product_id,
           product_name,
           ingresos,
           SUM(ingresos) OVER (ORDER BY ingresos DESC) as ingresos_acumulados,
           SUM(ingresos) OVER () as ingresos_totales
    FROM IngresosPorProducto
)
SELECT product_name,
       ingresos,
       ingresos_acumulados,
       ROUND((ingresos_acumulados * 100.0 / ingresos_totales), 2) as porcentaje_acumulado
FROM IngresosAcumulados
WHERE ingresos_acumulados <= ingresos_totales * 0.8 
   OR (ingresos_acumulados - ingresos) < ingresos_totales * 0.8
ORDER BY ingresos DESC
"""

resultado = ejecutar(consulta)
print("Producto | Ingresos | Ingresos acumulados | % acumulado\n")
for producto, ingresos, acum, porcentaje in resultado:
    print(f'{producto} | ${ingresos:.2f} | ${acum:.2f} | {porcentaje}%')
print(f"\nTotal de productos necesarios: {len(resultado)}")


Producto | Ingresos | Ingresos acumulados | % acumulado

Trek Slash 8 27.5 - 2016 | $555558.61 | $555558.61 | 7.23%
Trek Conduit+ - 2016 | $389248.70 | $944807.31 | 12.29%
Trek Fuel EX 8 29 - 2016 | $368472.73 | $1313280.04 | 17.08%
Surly Straggler 650b - 2016 | $226765.55 | $1540045.59 | 20.03%
Trek Domane SLR 6 Disc - 2017 | $211584.62 | $1751630.21 | 22.78%
Surly Straggler - 2016 | $203507.62 | $1955137.83 | 25.43%
Trek Remedy 29 Carbon Frameset - 2016 | $203380.87 | $2158518.70 | 28.07%
Trek Powerfly 8 FS Plus - 2017 | $188249.62 | $2346768.32 | 30.52%
Trek Madone 9.2 - 2017 | $175899.65 | $2522667.97 | 32.81%
Trek Silque SLR 8 Women's - 2017 | $174524.73 | $2697192.70 | 35.08%
Trek Silque SLR 7 Women's - 2017 | $163079.73 | $2860272.43 | 37.2%
Trek Fuel EX 9.8 27.5 Plus - 2017 | $159052.70 | $3019325.13 | 39.27%
Heller Shagamaw Frame - 2016 | $151160.89 | $3170486.02 | 41.23%
Trek Fuel EX 9.8 29 - 2017 | $147449.71 | $3317935.72 | 43.15%
Trek Boone 7 - 2017 | $127609.64 | $3445545

In [93]:
# Validación con Python
items_ingresos = order_items_df[['product_id', 'quantity', 'list_price', 'discount']].copy()
items_ingresos['ingresos'] = items_ingresos['quantity'] * items_ingresos['list_price'] * (1 - items_ingresos['discount'])
ingresos_por_producto = items_ingresos.groupby('product_id')['ingresos'].sum().reset_index()
ingresos_por_producto = ingresos_por_producto.merge(products_df[['product_id', 'product_name']], on='product_id')
ingresos_por_producto = ingresos_por_producto.sort_values('ingresos', ascending=False)
ingresos_totales = ingresos_por_producto['ingresos'].sum()
ingresos_por_producto['ingresos_acumulados'] = ingresos_por_producto['ingresos'].cumsum()
ingresos_por_producto['porcentaje_acumulado'] = (ingresos_por_producto['ingresos_acumulados'] / ingresos_totales) * 100
productos_80 = ingresos_por_producto[ingresos_por_producto['porcentaje_acumulado'] <= 80]
print("Validación Python - Productos que representan 80% del ingreso:")
display(productos_80[['product_name', 'ingresos', 'ingresos_acumulados', 'porcentaje_acumulado']].head(20))
print(f"\nTotal de productos necesarios: {len(productos_80)}")


Validación Python - Productos que representan 80% del ingreso:


,product_name,ingresos,ingresos_acumulados,porcentaje_acumulado
5,Trek Slash 8 27.5 - 2016,555558.6111,5.555586e+05,7.225259
7,Trek Conduit+ - 2016,389248.7025,9.448073e+05,12.287593
2,Trek Fuel EX 8 29 - 2016,368472.7294,1.313280e+06,17.079726
9,Surly Straggler 650b - 2016,226765.5510,1.540046e+06,20.028902
54,Trek Domane SLR 6 Disc - 2017,211584.6153,1.751630e+06,22.780643
8,Surly Straggler - 2016,203507.6200,1.955138e+06,25.427340
6,Trek Remedy 29 Carbon Frameset - 2016,203380.8701,2.158519e+06,28.072389
59,Trek Powerfly 8 FS Plus - 2017,188249.6235,2.346768e+06,30.520650
56,Trek Madone 9.2 - 2017,175899.6482,2.522668e+06,32.808294
49,Trek Silque SLR 8 Women's - 2017,174524.7315,2.697193e+06,35.078057



Total de productos necesarios: 68


Para finalizar, cerramos la conexión.


In [94]:
connection.close()
